Agora além de detectarmos faces, iremos também classificar os rostos de pessoas.

Iremos utilizar um Dataset da George Tech, onde temos 15 imagens por pessoa.

[Download Dataset](https://drive.google.com/file/d/1BTY76SQslUDvZ1eslC41Jre3LHkGGsZ_/view?usp=sharing)

In [ ]:
# TODO Identificar os rostos de cada imagem, cortar e salvar em memória (em um array) para o pré processamento.